### In this Part of the assignment, we create various models to predict the Winner of a race and evaluate the models using a testing dataset . 

In [52]:
# Create Training DataSet and Model Testing Dataset of Results DataFrame 

import pandas
from pandas import read_csv 
import numpy as np 
import matplotlib.pyplot as plt  

race_results = read_csv('results_full2(+pit_stops).csv')
race_results.dtypes

Unnamed: 0           int64
resultId             int64
raceId               int64
driverId             int64
constructorId        int64
grid                 int64
positionText        object
positionOrder        int64
points             float64
laps                 int64
fastestLapTime      object
fastestLapSpeed     object
status              object
driverRef           object
dob                 object
nationality_x       object
year                 int64
round                int64
date                object
circuitRef          object
location            object
country             object
constructorRef      object
nationality_y       object
position_quali     float64
stop                 int64
lap                  int64
milliseconds         int64
dtype: object

In [80]:
#transform categorical data to numerical before analysis
race_results["circuitRef"] = race_results["circuitRef"].astype('category') 
race_results["circuitRef"] = race_results["circuitRef"].cat.codes
race_results["driverRef"] = race_results["driverRef"].astype('category') 
race_results["driverRef"] = race_results["driverRef"].cat.codes
race_results["location"] = race_results["location"].astype('category') 
race_results["location"] = race_results["location"].cat.codes
race_results["country"] = race_results["country"].astype('category') 
race_results["country"] = race_results["country"].cat.codes
race_results["constructorRef"] = race_results["constructorRef"].astype('category') 
race_results["constructorRef"] = race_results["constructorRef"].cat.codes
race_results["nationality_y"] = race_results["nationality_y"].astype('category') 
race_results["nationality_y"] = race_results["nationality_y"].cat.codes
race_results["nationality_x"] = race_results["nationality_x"].astype('category') 
race_results["nationality_x"] = race_results["nationality_x"].cat.codes
race_results["dob"] = race_results["dob"].astype('category') 
race_results["dob"] = race_results["dob"].cat.codes
race_results["status"] = race_results["status"].astype('category') 
race_results["status"] = race_results["status"].cat.codes
race_results=race_results[~race_results.fastestLapTime.str.contains("\N")]
race_results=race_results[~race_results.fastestLapSpeed.str.contains("\N")]
race_results.head()

,Unnamed: 0,resultId,raceId,driverId,constructorId,grid,positionText,positionOrder,points,laps,...,date,circuitRef,location,country,constructorRef,nationality_y,position_quali,stop,lap,milliseconds
0,0,20780,841,1,1,2,2,2,18.0,58,...,3/27/2011,1,9,0,10,2,2.0,1,16,23227
1,1,20780,841,1,1,2,2,2,18.0,58,...,3/27/2011,1,9,0,10,2,2.0,2,36,23199
2,2,20784,841,18,1,4,6,6,8.0,58,...,3/27/2011,1,9,0,10,2,4.0,1,17,16867
3,3,20784,841,18,1,4,6,6,8.0,58,...,3/27/2011,1,9,0,10,2,4.0,2,19,23303
4,4,20784,841,18,1,4,6,6,8.0,58,...,3/27/2011,1,9,0,10,2,4.0,3,37,22681


In [81]:
#splitting the dataset into the Predictors (independant variables) and the Result variable (dependant variable)
predictors = race_results.drop('positionOrder', axis =1) 
predictors = race_results.drop('date', axis =1)#all columns except the results column
PositionOrder = race_results[['positionOrder']] #only the Position column
predictors



,Unnamed: 0,resultId,raceId,driverId,constructorId,grid,positionText,positionOrder,points,laps,...,round,circuitRef,location,country,constructorRef,nationality_y,position_quali,stop,lap,milliseconds
0,0,20780,841,1,1,2,2,2,18.0,58,...,1,1,9,0,10,2,2.0,1,16,23227
1,1,20780,841,1,1,2,2,2,18.0,58,...,1,1,9,0,10,2,2.0,2,36,23199
2,2,20784,841,18,1,4,6,6,8.0,58,...,1,1,9,0,10,2,4.0,1,17,16867
3,3,20784,841,18,1,4,6,6,8.0,58,...,1,1,9,0,10,2,4.0,2,19,23303
4,4,20784,841,18,1,4,6,6,8.0,58,...,1,1,9,0,10,2,4.0,3,37,22681
5,5,21234,860,1,1,1,3,3,15.0,58,...,1,1,9,0,10,2,1.0,1,17,22862
6,6,21234,860,1,1,1,3,3,15.0,58,...,1,1,9,0,10,2,1.0,2,36,23464
7,7,21232,860,18,1,2,1,1,25.0,58,...,1,1,9,0,10,2,2.0,1,16,23275
8,8,21232,860,18,1,2,1,1,25.0,58,...,1,1,9,0,10,2,2.0,2,36,22837
9,9,21720,880,18,1,10,9,9,2.0,58,...,1,1,9,0,10,2,10.0,1,4,22282


In [82]:
PositionOrder.head()

,positionOrder
0,2
1,2
2,6
3,6
4,6


In [83]:
# Compute Information Gain of each feauture and select only features with GINI greater than 70 

from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn import datasets

svm = LinearSVC()
# create the RFE model for the svm classifier 
# and select attributes
rfe = RFE(svm, 28)
rfe = rfe.fit(predictors,PositionOrder)
# print summaries for the selection of attributes
print(rfe.support_)
print(rfe.ranking_)


ValueError: invalid literal for float(): 01:37.0